GraphRAG with Networkx. Getting Trump to like peanut butter.

Stephan Raaijmakers, LUCL, 01.04.2025

In [108]:
!pip install --upgrade --quiet  json-repair networkx langchain-core  langchain-experimental langchain-community langchain_huggingface langchain_openai


In [109]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
import networkx as nx
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_openai import ChatOpenAI
from google.colab import userdata

In [110]:
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [111]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    #repo_id="google/gemma-7b",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    temperature=0.01
)

In [112]:
text = """
Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who is the 47th president of the United States.
A member of the Republican Party, he served as the 45th president from 2017 to 2021.
Born in New York City, Trump graduated from the University of Pennsylvania in 1968 with a bachelor's degree in economics.
He became the president of his family's real estate business in 1971, renamed it the Trump Organization, and began acquiring and building
skyscrapers, hotels, casinos, and golf courses. After six business bankruptcies in the 1990s and 2000s, he began side ventures.
From 2004 to 2015, he hosted the reality television show The Apprentice. A political outsider, Trump won the 2016 presidential election against
Democratic nominee Hillary Clinton. He likes peanut butter a lot on his sandwiches.
"""

In [113]:
documents = [Document(page_content=text)]
llm_graph_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_graph_transformer.convert_to_graph_documents(documents)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


In [114]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='47th president of the United States', type='Politician', properties={}), Node(id='The Apprentice', type='TelevisionShow', properties={}), Node(id='New York City', type='Location', properties={}), Node(id='peanut butter', type='Food', properties={}), Node(id='June 14, 1946', type='Date', properties={}), Node(id='45th president of the United States', type='Politician', properties={}), Node(id='Donald John Trump', type='Person', properties={}), Node(id='University of Pennsylvania', type='EducationalInstitution', properties={}), Node(id='Trump Organization', type='Company', properties={})]
Relationships:[Relationship(source=Node(id='Donald John Trump', type='Person', properties={}), target=Node(id='New York City', type='Location', properties={}), type='BORN_IN', properties={}), Relationship(source=Node(id='Donald John Trump', type='Person', properties={}), target=Node(id='University of Pennsylvania', type='EducationalInstitution', properties={}), type='GRADUATED_FROM', prop

In [115]:
# Here, we are using OpenAI. Can you find a Huggingface mode that also works?
# If so, then adapt llm=HuggingfacePoint(...) adapt and comment out this next line!
llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo",openai_api_key=OPENAI_API_KEY)

llm_graph_transformer_selection = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person",
                  "Food"],
    allowed_relationships=["LIKES"],
)
graph_documents_selection = llm_graph_transformer_selection.convert_to_graph_documents(
    documents
)

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1637: UserWarning: Cannot use method='json_schema' with model gpt-4-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [116]:
print(f"Nodes:{graph_documents_selection[0].nodes}")
print(f"Relationships:{graph_documents_selection[0].relationships}")

Nodes:[Node(id='Donald John Trump', type='Person', properties={}), Node(id='Hillary Clinton', type='Person', properties={}), Node(id='Peanut Butter', type='Food', properties={})]
Relationships:[Relationship(source=Node(id='Donald John Trump', type='Person', properties={}), target=Node(id='Peanut Butter', type='Food', properties={}), type='LIKES', properties={})]


In [117]:
graph = NetworkxEntityGraph()

# Add nodes
for node in graph_documents_selection[0].nodes:
    graph.add_node(node.id)

# Add edges
for edge in graph_documents_selection[0].relationships:
    graph._graph.add_edge(
            edge.source.id,
            edge.target.id,
            relation=edge.type,
        )

In [118]:
chain = GraphQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True
)

In [119]:
question = """What food does Donald John Trump like?"""
print(chain.run(question))



> Entering new GraphQAChain chain...
Entities Extracted:
Donald John Trump
Full Context:
Donald John Trump LIKES Peanut Butter

> Finished chain.
Donald John Trump likes peanut butter.
